# Face Recognition Using Sianese Network

In [1]:
# dependencies
!pip install opencv-python matplotlib scikit-learn pandas


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
from itertools import combinations
import csv
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
import tensorflow as tf
from tensorflow.keras import layers, Model


KeyboardInterrupt



In [9]:
PAIRS = []
ROOT = './lfw'

### Create Pairs for the Triplet Loss

In [ ]:
def create_similarity_pairs():
    list_dir = list(os.listdir(ROOT))
    output = []
    
    for i, faces in enumerate(list_dir):
        file_path = os.path.join(ROOT, faces)
        for pair in combinations(os.listdir(file_path), 2):
            output.append([faces, pair[0], faces, pair[1], 1])

    return output;

In [ ]:
def create_dissimilarity_pairs():
    list_dir = list(os.listdir(ROOT))
    output = []

    for i, faces_name in enumerate(list_dir):
        list_face1 = os.listdir(os.path.join(ROOT, faces_name))
        
        for j in range(i + 1, min(i+40, len(list_dir))):
            list_face2 = os.listdir(os.path.join(ROOT, list_dir[j]))
            output.append([faces_name, list_face1[0], list_dir[j], list_face2[0], 0])
        
    return output

In [ ]:
def create_pairs():
    return create_similarity_pairs() + create_dissimilarity_pairs()

In [ ]:
PAIRS = create_pairs()

In [ ]:
PAIRS[-1]

# Convert the data into csv

In [11]:
# Convert the result of pairs into csv
with open('dataset.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(PAIRS)


In [5]:
image_df = pd.read_csv('./dataset.csv')
image_df.columns = ['input1_name', 'input1_filename', 'input2_name', 'input2_filename', 'similarity']
image_df.head()

,input1_name,input1_filename,input2_name,input2_filename,similarity
0,Aaron_Peirsol,Aaron_Peirsol_0001.jpg,Aaron_Peirsol,Aaron_Peirsol_0003.jpg,1
1,Aaron_Peirsol,Aaron_Peirsol_0001.jpg,Aaron_Peirsol,Aaron_Peirsol_0004.jpg,1
2,Aaron_Peirsol,Aaron_Peirsol_0002.jpg,Aaron_Peirsol,Aaron_Peirsol_0003.jpg,1
3,Aaron_Peirsol,Aaron_Peirsol_0002.jpg,Aaron_Peirsol,Aaron_Peirsol_0004.jpg,1
4,Aaron_Peirsol,Aaron_Peirsol_0003.jpg,Aaron_Peirsol,Aaron_Peirsol_0004.jpg,1


### Read image

In [6]:
def preprocess_image(img, target_size):
    img = cv2.resize(img, target_size)
    img = img / 255.0
    return img

In [7]:
def read_and_preprocess_images(df, target_size=(180, 180)):
    images1 = []
    images2 = []
    similarity = df['similarity'].to_numpy()
    for index, row in df.iterrows():
        path1 = os.path.join(ROOT, row['input1_name'], row['input1_filename'])
        path2 = os.path.join(ROOT, row['input2_name'], row['input2_filename'])
        
        img1 = cv2.imread(path1)
        img2 = cv2.imread(path2)

        img1 = preprocess_image(img1, target_size)
        img2 = preprocess_image(img2, target_size)
            
        images1.append(img1)
        images2.append(img2)
    
    return np.array(images1), np.array(images2), siimilarity

In [ ]:
input1, input2, similarity = read_and_preprocess_images(image_df)

### Build the Model

In [ ]:


# Define the base network
def create_base_network(input_shape):
    input = layers.Input(shape=input_shape)
    x = layers.Conv2D(64, (3, 3), activation='relu')(input)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(128, (3, 3), activation='relu')(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Flatten()(x)
    x = layers.Dense(128, activation='relu')(x)
    return Model(input, x)

input_shape = input1_images.shape[1:]  # Assuming all images have the same shape
base_network = create_base_network(input_shape)

# Create the inputs
input_a = layers.Input(shape=input_shape)
input_b = layers.Input(shape=input_shape)

# Generate the feature vectors for the two images
processed_a = base_network(input_a)
processed_b = base_network(input_b)

# Compute the Euclidean distance between the feature vectors
distance = layers.Lambda(lambda tensors: tf.sqrt(tf.reduce_sum(tf.square(tensors[0] - tensors[1]), axis=1, keepdims=True)))(
    [processed_a, processed_b])

# Define the model
model = Model([input_a, input_b], distance)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


# Split the test and train test

In [40]:
# Perform the train-test split
train_df, test_df = train_test_split(image_df, test_size=0.2, stratify=image_df['similarity'], random_state=42)
